In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Loading, scaling the data
data
has
been
explored and concaternated in `data_preparerationipynb`
# Load the dataset
df = pd.read_csv('../data/dataset_full.csv')
df.shape

# Separate the target variable from the features
X = df.drop('price_range', axis=1)
y = df['price_range']

#  Split the data into train, validation, and test sets 0.7, 0.15, 0.15
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Create scaling pipeline for the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [ ]:
# https://www.tensorflow.org/tutorials/keras/regression